In [1]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import os
import openpyxl as px
import xlwings as xw
import numpy as np
import psycopg2 as pg
import configparser
import shutil

In [2]:
#path premissas 
path_premissas = r'F:\Silvicultura\01. SP\PROCESSOS\COI Silvicultura - Edicao\01. Programações & Controles\001. Rolling Forcast - RF\2024\Distribuição Operacional\premissas.xlsx'

In [3]:
premissas_df = pd.read_excel(path_premissas, sheet_name='path')

def input_file(file, name_path):
    premissas = premissas_df.copy()
    
    premissas = premissas[premissas['status'] == 'ativo']

    premissas = premissas[premissas['arquivos de consumo'] == file]

    path = premissas['path'].iloc[0]

    return path


In [4]:
#path credenciais

credenciais_path = input_file(file='credenciais_path', name_path='path')

In [5]:
#credenciais


credenciais = configparser.ConfigParser()

credenciais.read(credenciais_path)

logins = credenciais['credenciais']


conn_dwh = pg.connect(
    user =logins['login'],
    password =logins['senha_sql'],
    host = logins['host'],
    port = logins['port'],
    database = logins['data_base']
)

In [6]:
#path histórico

path_historico = input_file(file='path_historico', name_path='path')

In [7]:
#path lista tecnica

path_lista_tecnica = input_file(file='path_lista_tecnica', name_path='path')

In [8]:
#Path general service

path_general_service = input_file(file='path_general_service', name_path='path')

In [9]:
#Base de nomenclatura de operações

general_service = pd.read_excel(path_general_service, sheet_name='APOIO', usecols=['General Services','Others'])

#Removendo duplicatas
general_service = general_service.drop_duplicates(['General Services'], keep='first')

In [10]:
#base histórico realizado
historico = pd.read_excel(path_historico, skiprows=2)

In [11]:
#Capacidade de plantio
cap_plantio = pd.read_excel(path_premissas,sheet_name='cap plantio')

In [12]:
#Encontrando mês operacional

def mes_operacional(df, coluna, nome_coluna):
    # Garantir que a coluna é do tipo datetime
    if not pd.api.types.is_datetime64_any_dtype(df[coluna]):
        df[coluna] = pd.to_datetime(df[coluna], errors='coerce')
    
    
    def calcular_mes_operacional(data):
        
        if pd.isna(data):
            return None 
        
        
        dia = data.day
        mes = data.month
        ano = data.year
        
        
        if dia > 20:
            mes = (mes % 12) + 1
            if mes == 1:
                ano += 1
        
        # Retornar um Timestamp, garantindo que todos são inteiros
        return pd.Timestamp(year=int(ano), month=int(mes), day=1)

    # Aplicar a função para a coluna
    df[nome_coluna] = df[coluna].apply(calcular_mes_operacional)

    return df

In [13]:
#Código de usuário responsável


cursor_usu_resp = conn_dwh.cursor()

cursor_usu_resp.execute(
""" 
select distinct on (cd_usuario)
    dcr_usuario,
    cd_usuario,
    est_reg

from 
    forestry.d_vw_usuarios_perfil
"""

)

#Mesmo o est_reg for apenas 'A' ele considera as pessoas que já não estão mais na empresa


dados_uso_resp = cursor_usu_resp.fetchall()

cursor_usu_resp.close()


colunas_uso_resp = [desc[0] for desc in cursor_usu_resp.description]

#Fechando conexão com o DWH
conn_dwh.close()

uso_resp = pd.DataFrame(dados_uso_resp, columns=colunas_uso_resp)


In [14]:
#Inserindo Supervisor

historico = historico.merge(uso_resp[['dcr_usuario','cd_usuario']],left_on=['cd_usuario_resp'], right_on= ['cd_usuario'], how='left').drop('cd_usuario',axis=1)

In [15]:
#Filtrando somente o que é plantio
historico = historico[historico['Grupo de Atividades'] == 'Plantio']

In [16]:
#De para EPS

de_para_eps = pd.read_excel(path_premissas, sheet_name='eps')

In [17]:
#Encontrando EPS JFI

historico['os_eps'] = historico.apply(lambda x : str(x['dcr_ordem_servico'])[8:].split('_')[1].upper(), axis=1)

historico['dcr_fornecedor'] = np.where(historico['dcr_fornecedor'] == 'J F I SILVICULTURA', historico['os_eps'], historico['dcr_fornecedor'])

historico['dcr_fornecedor'] = np.where( (historico['os_eps'] == 'JFI') & (historico['dcr_usuario'] == 'Danilo Roberto Bavaroski') , 'JFICAPÃO', historico['dcr_fornecedor'])

In [18]:
#retirando do histórico os meses que ainda estão planejados e não estão fechado

# historico = historico[(historico['[m] Mês Operacional'] != '2024_11') & (historico['[m] Mês Operacional'] != '2024_12')]

# historico = historico[(historico['[m] Mês Operacional'] != '2024_12')]

In [19]:
#de_para eps
historico = historico.merge(de_para_eps, left_on=['dcr_fornecedor'], right_on=['eps_de'],how='left').drop('eps_de',axis=1).rename({'eps_para': 'Equipe de Plantio'},axis=1)

In [20]:
#Export histórico
historico.to_excel('l - historico.xlsx')

In [21]:
#V_anual_pivot

v_anual = historico.groupby('[m] Mês Operacional')['Soma de [c] Valor de Produção'].sum().reset_index()


#Alterando o tipo de mês operacional

v_anual['[m] Mês Operacional'] = pd.to_datetime(v_anual['[m] Mês Operacional'], format= '%Y_%m')


#Volume anual pivot, comparar com os próximos anos
v_anual_pivot = pd.pivot_table(v_anual, values=['Soma de [c] Valor de Produção'], columns=['[m] Mês Operacional'])


In [22]:
#v_anual_eps

v_anual_eps = historico.groupby(['[m] Mês Operacional','dcr_fornecedor','Equipe de Plantio'])['Soma de [c] Valor de Produção'].sum().reset_index()

#Encontrar proporção de plantio 
v_anual_eps['% de Plantio'] = v_anual_eps['Soma de [c] Valor de Produção'] / v_anual_eps.groupby('[m] Mês Operacional')['Soma de [c] Valor de Produção'].transform('sum')


In [23]:
#v_anual_eps pivot

#Visão de volume por eps ao longo dos anos
v_anual_eps_pivot = v_anual_eps.pivot_table(index=['Equipe de Plantio'], columns=['[m] Mês Operacional'], values='Soma de [c] Valor de Produção',aggfunc='sum')

#Visão em percentual por eps ao longo dos anos
perc_anual_eps_pivot = v_anual_eps.pivot_table(index=['Equipe de Plantio'], columns=['[m] Mês Operacional'], values='% de Plantio',aggfunc='sum')

In [24]:
#Export pivot anual_eps
#perc_anual_eps_pivot.to_excel('l - percentual de plantio.xlsx')
#v_anual_eps_pivot.to_excel('l - volume de plantio.xlsx')

In [25]:
#cap_plantio_melt



cap_plantio_melt = cap_plantio.melt(id_vars=['Service Provider'], 
                                    value_vars=[ '2025_01', '2025_02',
                                                '2025_03', '2025_04', '2025_05', '2025_06', '2025_07', '2025_08',
                                                '2025_09', '2025_10', '2025_11', '2025_12', '2026_01', '2026_02',
                                                '2026_03', '2026_04', '2026_05'],
                                    var_name='[m] Mês Operacional',
                                    value_name='Soma de [c] Valor de Produção',
                                    )


#Renomear coluna
cap_plantio_melt = cap_plantio_melt.rename({'Service Provider': 'Equipe de Plantio'}, axis=1)

#Alterando o tipo de dado númerico
cap_plantio_melt['Soma de [c] Valor de Produção'] = pd.to_numeric(cap_plantio_melt['Soma de [c] Valor de Produção'], errors = 'coerce')

In [26]:
#Concatenando base para rolling_forcast

rolling_forcast = pd.concat([v_anual_eps,
                             cap_plantio_melt])

In [27]:
#Atribuindo regional de acordo com a equipe
rolling_forcast = rolling_forcast.merge(de_para_eps[['eps_para','regional','classe equipe']].drop_duplicates('eps_para'),left_on=['Equipe de Plantio'], right_on=['eps_para'],how='left').drop('eps_para',axis=1)


#Encontrar proporção de plantio 
rolling_forcast['% de Plantio'] = rolling_forcast['Soma de [c] Valor de Produção'] / rolling_forcast.groupby('[m] Mês Operacional')['Soma de [c] Valor de Produção'].transform('sum')


In [28]:
#Colunas utilizadas na Lista técnica de operações
cols_lista_tecnica = ['Fase custo padrão','Time (Days)','Operation Description','Tipo','Intensity',
                      'Jan-INT', 'Fev-INT', 'Mar-INT', 'Abr-INT',
                      'Mai-INT', 'Jun-INT', 'Jul-INT', 'Ago-INT', 
                      'Set-INT', 'Out-INT', 'Nov-INT', 'Dez-INT']

In [29]:
#Lista técnica regional SU

lista_tecnica_su = pd.read_excel(path_lista_tecnica,sheet_name='LT_REFSU',skiprows=1, usecols=cols_lista_tecnica)

lista_tecnica_su = lista_tecnica_su[lista_tecnica_su['Tipo'] == 'Serviço']

lista_tecnica_su['regional'] = 'SU'

In [30]:
#Lista técnica regional CE

lista_tecnica_ce = pd.read_excel(path_lista_tecnica,sheet_name='LT_REFCE',skiprows=1, usecols=cols_lista_tecnica)

lista_tecnica_ce = lista_tecnica_ce[lista_tecnica_ce['Tipo'] == 'Serviço']

lista_tecnica_ce['regional'] = 'CE'

In [31]:
#Lista técnica regional NW

lista_tecnica_nw = pd.read_excel(path_lista_tecnica,sheet_name='LT_REFNR',skiprows=1, usecols=cols_lista_tecnica)

lista_tecnica_nw = lista_tecnica_nw[lista_tecnica_nw['Tipo'] == 'Serviço']

lista_tecnica_nw['regional'] = 'NW'

In [32]:
#Lista técnica regional NO

lista_tecnica_no = pd.read_excel(path_lista_tecnica,sheet_name='LT_REFNO',skiprows=1, usecols=cols_lista_tecnica)

lista_tecnica_no = lista_tecnica_no[lista_tecnica_no['Tipo'] == 'Serviço']

lista_tecnica_no['regional'] = 'NO'

In [33]:
#Lista técnica regional CP
    #Talhadia

lista_tecnica_cp = pd.read_excel(path_lista_tecnica,sheet_name='LT_REBGL',skiprows=1, usecols=cols_lista_tecnica)

lista_tecnica_cp = lista_tecnica_cp[lista_tecnica_cp['Tipo'] == 'Serviço']

lista_tecnica_cp['regional'] = 'CP'

In [34]:
#Concatenando lista tecnica

lista_tecnica = pd.concat([lista_tecnica_su,
                          lista_tecnica_ce,
                          lista_tecnica_no,
                          lista_tecnica_nw,
                          lista_tecnica_cp])


#Removendo fase Harvesting devido por se repetir na fase maintenance 5-6
lista_tecnica = lista_tecnica[lista_tecnica['Fase custo padrão'] != 'HARVESTING']

In [35]:
#Export lista técnica
lista_tecnica.to_excel('l - lista_tecnica.xlsx')

In [36]:
#Cadastro de operações

cad_op = pd.read_excel(path_premissas,sheet_name='cad operacoes')

In [37]:
#Atribuindo equipes a lista tecnica

#Não posso atribuir equipe a lista técnica, seria um erro pensar que todas as equipes fazem todas as operações, o que não é o caso

#lista_tecnica = lista_tecnica.merge(de_para_eps[['eps_para','regional']].drop_duplicates('eps_para'), left_on='regional', right_on='regional', how='left')

In [38]:
#Alterando o formato de datas rolling_forcast
rolling_forcast['[m] Mês Operacional'] = pd.to_datetime(rolling_forcast['[m] Mês Operacional'],format='%Y_%m')

In [39]:
#Produção bdsgf

#Área indicada pelo Caio

bdsgf = pd.read_excel(path_premissas, sheet_name='area referencia')

bdsgf = bdsgf[bdsgf['Tipo'] != 'Exaustão']

bdsgf = bdsgf.groupby(['Mês'])['Área'].sum().reset_index()

bdsgf_pivot = pd.pivot_table(bdsgf,values='Área',columns=['Mês'])

bdsgf_pivot = bdsgf_pivot.rename({'Área':'Área bdsgf'},axis=0)


In [40]:
#Renomeando v_anual_pivot
v_anual_pivot = v_anual_pivot.rename({'Soma de [c] Valor de Produção':'Área sgf'}, axis = 0)


In [41]:
#delta_sgf_bdsgf

#Encontrando delta entre o que foi apontado no sgf e o que foi utilizado no orçamento

#Verificar volume de plantio apontado no sgf vs apontado no cadastro de áreas budget

#E ajustar conforme volume plantado, remover organicamente os percentuais de acordo com o mes e ano, chamaremos de delta ajustado bdsgf


delta_sgf_bdsgf = pd.concat([v_anual_pivot,bdsgf_pivot])

delta_sgf_bdsgf.loc['delta ha'] = delta_sgf_bdsgf.loc['Área bdsgf'] - delta_sgf_bdsgf.loc['Área sgf']

delta_sgf_bdsgf.loc['delta %'] = delta_sgf_bdsgf.loc['Área bdsgf'] / delta_sgf_bdsgf.loc['Área sgf']

delta_sgf_bdsgf.loc['ajuste'] = delta_sgf_bdsgf.loc['delta %'] - 1

In [42]:
#Export delta sgf
#delta_sgf_bdsgf.to_excel("l - delta_sgf_bdsgf.xlsx")

In [43]:
#Transpondo o Data Frame para encontrar o ajuste necessário

t_delta_sgf_bdsgf = delta_sgf_bdsgf.T.reset_index()

t_delta_sgf_bdsgf = t_delta_sgf_bdsgf.rename({'index': 'Mês Operacional'},axis=1)

In [44]:
#Atribuir os ajustes no rolling forecast

rolling_forcast = rolling_forcast.merge(t_delta_sgf_bdsgf[['Mês Operacional', 'delta ha']], left_on=['[m] Mês Operacional'], right_on=['Mês Operacional'], how='left').drop('Mês Operacional', axis=1)


In [45]:
#Ajustando volumes rolling forcast

#Pegaremos os valores de delta (ha) e o percentual que cada eps plantou

rolling_forcast['Soma de [c] Valor de Produção'] = np.where(pd.notna(rolling_forcast['delta ha']),

((rolling_forcast['% de Plantio'] * rolling_forcast['delta ha']) + rolling_forcast['Soma de [c] Valor de Produção']), rolling_forcast['Soma de [c] Valor de Produção'])


In [46]:
#Alteração mês de março
#Devido ao calendário operacional

#Antes de atribuir a lista técnica, alterar o mês de março para dia 02 (*Observações ao mês de fevereiro)

rolling_forcast['[m] Mês Operacional'] = np.where(rolling_forcast['[m] Mês Operacional'].dt.month == 3, 
                                                rolling_forcast['[m] Mês Operacional'] + timedelta(days= 2),
                                                rolling_forcast['[m] Mês Operacional'])

In [47]:
#Atribuindo a lista tecnica ao rolling_forcast

rolling_forcast = rolling_forcast.merge(lista_tecnica,left_on=['regional'], right_on=['regional'], how='left')

In [48]:
#Export rolling forcast - antes de ajustes
#rolling_forcast.to_excel('l - rolling_forcast.xlsx', index=False)

In [49]:
#Atribuindo operações

rolling_forcast = rolling_forcast.merge(cad_op, left_on='Operation Description', right_on='Operações', how='left').drop('Operações',axis=1)

In [50]:
#Atribuir quais equipes irão executar atividade

#Atribuindo bracell 1 para operações que não devem ser feitas de forma tercerizada

#Atribuir terceiros para operações que bracell atual
    #Pensar como redistribuir as operações definidas para terceiros


#Classe de trabalho -> Tipo de operação e qual frente deve atuar

#Classe equipe -> Se a equipe é própria ou terceira


rolling_forcast['Equipe de Plantio - R'] = np.where((rolling_forcast['classe equipe'] == 'Terceiros') &
                                                    (rolling_forcast['Classe de trabalho'] == 'Bracell')
                                                    ,'Bracell 1',
                                                    
                                                    np.where( (rolling_forcast['classe equipe'] == 'Bracell') & 
                                                            (rolling_forcast['Classe de trabalho'] == 'Terceiros'),'Terceiros',
                                                    
                                                        np.where(((rolling_forcast['classe equipe'] == 'Terceiros') |
                                                                    (rolling_forcast['classe equipe'] == 'Bracell') |
                                                                    (rolling_forcast['classe equipe'] == 'Coppice')) &
                                                                    (rolling_forcast['Classe de trabalho'] == 'Bracell Apoio'), 'Bracell Apoio',
                                                                    rolling_forcast['Equipe de Plantio'])))


In [51]:
#Encontrando as datas de operação 

rolling_forcast['data de operação'] = rolling_forcast['[m] Mês Operacional'] + pd.to_timedelta(rolling_forcast['Time (Days)'], unit='D')

#Mês operacional

rolling_forcast = mes_operacional(rolling_forcast,'data de operação', 'data de operação - co')


In [52]:
#Intensidade e Sazonalidade

#Encontrando sazonalidade e intensidade de execusão de atividade

rolling_forcast['v_intensidade'] = np.where(
    rolling_forcast['data de operação - co'].dt.month == 1, rolling_forcast['Soma de [c] Valor de Produção'] * rolling_forcast['Jan-INT'],
    np.where(
        rolling_forcast['data de operação - co'].dt.month == 2, rolling_forcast['Soma de [c] Valor de Produção'] * rolling_forcast['Fev-INT'],
        np.where(
            rolling_forcast['data de operação - co'].dt.month == 3, rolling_forcast['Soma de [c] Valor de Produção'] * rolling_forcast['Mar-INT'],
            np.where(
                rolling_forcast['data de operação - co'].dt.month == 4, rolling_forcast['Soma de [c] Valor de Produção'] * rolling_forcast['Abr-INT'],
                np.where(
                    rolling_forcast['data de operação - co'].dt.month == 5, rolling_forcast['Soma de [c] Valor de Produção'] * rolling_forcast['Mai-INT'],
                    np.where(
                        rolling_forcast['data de operação - co'].dt.month == 6, rolling_forcast['Soma de [c] Valor de Produção'] * rolling_forcast['Jun-INT'],
                        np.where(
                            rolling_forcast['data de operação - co'].dt.month == 7, rolling_forcast['Soma de [c] Valor de Produção'] * rolling_forcast['Jul-INT'],
                            np.where(
                                rolling_forcast['data de operação - co'].dt.month == 8, rolling_forcast['Soma de [c] Valor de Produção'] * rolling_forcast['Ago-INT'],
                                np.where(
                                    rolling_forcast['data de operação - co'].dt.month == 9, rolling_forcast['Soma de [c] Valor de Produção'] * rolling_forcast['Set-INT'],
                                    np.where(
                                        rolling_forcast['data de operação - co'].dt.month == 10, rolling_forcast['Soma de [c] Valor de Produção'] * rolling_forcast['Out-INT'],
                                        np.where(
                                            rolling_forcast['data de operação - co'].dt.month == 11, rolling_forcast['Soma de [c] Valor de Produção'] * rolling_forcast['Nov-INT'],
                                            np.where(
                                                rolling_forcast['data de operação - co'].dt.month == 12, rolling_forcast['Soma de [c] Valor de Produção'] * rolling_forcast['Dez-INT'],
                                                
                                                rolling_forcast['Soma de [c] Valor de Produção']  # Valor padrão
                                            )
                                        )
                                    )
                                )
                            )
                        )
                    )
                )
            )
        )
    )
)





In [53]:
#Reprogramação

#Reprogramar volumes para equipes indicados pela aba de reprogramação

reprogramacao = pd.read_excel(path_premissas, sheet_name='reprogramacao')

#Removendo colunas que são de interesse para colunas e mantendo apenas linhas
cols_reprog_data = [col for col in reprogramacao.columns if col not in ['Operação', 'Equipe atual', 'Nova Equipe responsável', 'Tipo de valor']]

cols_reprog_index = ['Operação', 'Equipe atual', 'Nova Equipe responsável', 'Tipo de valor']


#melt de reprogramação
    #melt retira o pivot table e pivot table cria o melt

reprogramacao_melt = reprogramacao.melt(id_vars=cols_reprog_index , value_vars=cols_reprog_data, value_name='Valores', var_name='Mês Operacional')


#Retirando os percentuais vazios, para não duplicar a base no momento que fizer o merge com o rolling forecast
reprogramacao_melt = reprogramacao_melt[pd.notna(reprogramacao_melt['Valores'])]


#Alterando o tipo de dado do mês operacional

reprogramacao_melt['Mês Operacional'] = pd.to_datetime(reprogramacao_melt['Mês Operacional'], format='%Y_%m')



In [54]:
#Rolling forcast e reprogramação

rolling_forcast = rolling_forcast.merge(reprogramacao_melt[['Operação', 'Equipe atual', 'Nova Equipe responsável','Mês Operacional', 'Valores']], 
        left_on=['Equipe de Plantio - R', 'Operation Description', 'data de operação - co'],
        right_on=['Equipe atual', 'Operação', 'Mês Operacional'],
         how='left').drop(['Operação','Mês Operacional', 'Equipe atual'],axis=1)



#Alterando valores de equipe atual

rolling_forcast['Equipe de Plantio - R'] = np.where(pd.notna(rolling_forcast['Nova Equipe responsável']), 
                                                    rolling_forcast['Nova Equipe responsável'],
                                                    rolling_forcast['Equipe de Plantio - R'])


#Alterando valores de v_intensidade

rolling_forcast['v_intensidade'] = np.where(pd.notna(rolling_forcast['Valores']),
                                            (rolling_forcast['v_intensidade'] * rolling_forcast['Valores']),
                                            rolling_forcast['v_intensidade'])

In [55]:
#Rolling forcast após ajustes

#rolling_forcast.to_excel('e - rolling_forcast.xlsx', index=False)

In [56]:
#Condição de Área
#Definindo diferenças para Rebrota e Reforma

cond_map = {'CP': 'Rebrota',
            'CE': 'Reforma',
            'NO': 'Reforma',
            'NW': 'Reforma',
            'SU': 'Reforma'
            }

#Atribuindo a condição de área
rolling_forcast['Condição de Área'] = rolling_forcast['regional'].map(cond_map)

In [57]:
#Base query

#Agrupar volumes que foram duplicados

#Encontrar base para classificar operações da mesma forma que o PCP, olharemos para as mesmas operações dentro dos mesmos grupos

rf_agg = {'v_intensidade':'sum',
          #'Time (Days)': 'mean',
          }

rf_query = rolling_forcast.groupby(['Fase custo padrão','Operation Description','Condição de Área','Equipe de Plantio - R','data de operação - co']).agg(rf_agg).reset_index()


In [58]:
#Listando Regional apartir da EPS

rf_query = rf_query.merge(de_para_eps[['eps_para','classe equipe','regional']].drop_duplicates(['eps_para'],keep='first'), left_on='Equipe de Plantio - R', right_on='eps_para', how='left').drop(['eps_para'],axis=1)

In [59]:
#Merge de grupo de operações

rf_query = rf_query.merge(general_service,left_on=['Operation Description'], right_on=['General Services'], how='left').drop(['General Services'], axis = 1)


In [60]:
#Renomeando grupo de operações

rename_query_rf = {'Fase custo padrão': 'Fase Custo Padrão',
                   'Operation Description': 'Descrição de Operação',
                   'regional_x' : 'Lista Técnica',
                   'regional' : 'Regional',
                   'Equipe de Plantio - R': 'EPS',
                   '[m] Mês Operacional': 'Data De Plantio',
                   'data de operação - co' : 'Previsão de Data de Operação',
                   'v_intensidade' : 'Volume (Ha)',
                   'Time (Days)' : 'Tempo em relação ao Plantio (Dias)',
                   'Others':'Grupo de Operações',
                   'classe equipe': 'Classe Equipe'}



rf_query = rf_query.rename(rename_query_rf,axis=1)

In [61]:
#Definindo meses ativos

mes_ativo = pd.read_excel(path_premissas, sheet_name='periodo_ativo')

mes_ativo = mes_ativo.drop_duplicates('Mês Operacional', keep='first')

mes_ativo['Mês Operacional'] = pd.to_datetime(mes_ativo['Mês Operacional'], format= '%Y_%m')


In [62]:
#Filtrando apenas meses Sensíveis
#O envio para operação deve ser em formato de trimestres (*Devido a informações sensíveis)

#Filtro Ano 2025

rf_query = rf_query[rf_query['Previsão de Data de Operação'].dt.year == 2025]

#Filtro para retirar EPS Bracell Apoio
rf_query = rf_query[rf_query['EPS'] != 'Bracell Apoio']

#Adicionar filtro trimestral
rf_query = rf_query.merge(mes_ativo, left_on='Previsão de Data de Operação', right_on='Mês Operacional', how='left')


rf_query = rf_query[rf_query['Status Mês'] == 'Ativo']


#Remover operações < 0
#Isso incluí operações com intensidade 0 na lista tecnica

rf_query = rf_query[rf_query['Volume (Ha)'] > 0]


In [63]:
#Reordenar Colunas

rf_query = rf_query[['Fase Custo Padrão','Grupo de Operações','Descrição de Operação','Condição de Área','Regional','EPS','Classe Equipe',
                     'Previsão de Data de Operação','Volume (Ha)']]

In [64]:
#Puxando a variável para salvar o nome do arquivo

path_save_rf = input_file(file='path_save_rf', name_path = 'path')

In [65]:
#Puxando a variável para adicioanr o número do rf

path_number_rf = input_file(file='path_number_rf', name_path='path')

In [66]:
#Arquivo Modelo

path_model_file = input_file(file='path_model_file', name_path='path')

In [67]:
rf_query['EPS'].unique()

array(['Bracell 1', 'Equipe Escavadeira', 'Carpelo', 'Emflora', 'GN',
       'Geração', 'Gers', 'Inovesa', 'JFI Duartina', 'JFI Itatinga',
       'JFI Sul (CB/DO)', 'Migrar', 'Nilza', 'RA', 'Sollum', 'Tecponta',
       'Vemflors', 'ALSV', 'Maverick', 'Terceiros'], dtype=object)

In [68]:
#Criar função para exportar os dados para cada pasta e cada uma com seu próprio mês

#Desenho da estrutura:

# Rolling Forcast - RF
    #2024
        #RF01 > Budget > EPS
        #RF02 > Budget > EPS
        #RF12 > Budget > EPS
    
    #Definindo qual é o arquivo modelo e fazendo a cópia dele 


#Criar lógica da pasta e onde deve ser salvo


#Definir em qual pasta será salvo

def fechamento(path_folder_file, rf_number):

    eps_list = rf_query['EPS'].unique().tolist()

    
    for eps in eps_list:

        #Realizando filtro por EPS
        rf_subquery = rf_query[rf_query['EPS'] == eps]

        #Renomeando os itens que não podem conter no nome do excel        
        eps_rename = eps.replace('/' , '-')

        try:
            #Criando o diretório para salvar o arquivo
            dir_path = os.path.join(path_folder_file, "RF_"+str(rf_number),eps_rename)

            #Escolhendo o local para criação
            make_dir = os.makedirs(dir_path, exist_ok=True)

            #Definindo o nome do arquivo principal
            file_name_rf = os.path.basename(path_model_file)

            #Junção o novo nome
            file_name_rf_join = os.path.join(dir_path,file_name_rf)

            #Salvando arquivo do RF
            file_path = os.path.join(dir_path, f'RF_{rf_number}_{eps_rename}.xlsx')
            rf_subquery.to_excel(file_path,index = False)
            
            print(f"Exportando arquivo da {eps}")

            #Realizando a cópia do arquivo modelo
            shutil.copy(path_model_file,dir_path)

            
            #new name
            new_name = eps_rename + " - " + file_name_rf
            new_name_rf_join = os.path.join(dir_path, new_name)

            
            with xw.App(visible=True) as app:  # Definindo visibilidade para False
                workbook = app.books.open(file_name_rf_join)

                # Acessando a planilha 'path'
                worksheet = workbook.sheets['path']
                
                #Difinir o que será escrito na célula -> variável dir_path
                # Escrevendo o diretório na célula A2
                worksheet.range('A2').value = dir_path

                #Atualizar as consultas do Power Query
                workbook.api.RefreshAll()

                #Força a atualização
                worksheet.api.Calculate()

                # Salvando e fechando o arquivo
                workbook.save()
                
                #Fechando os arquivos
                workbook.close()


            #Renomear o arquivo com o nome da eps              
            os.rename(file_name_rf_join , new_name_rf_join)

      
        except Exception as e:
            # Se houver algum erro, ele será capturado aqui
            print(f"Erro ao criar o diretório: {e}")

        
        

    return print("Arquivos exportados com Sucesso")

    


In [69]:
#fechamento(path_save_rf, path_number_rf)

In [70]:
#Query RF export
#Criar lógica para exportar por mês e já criar uma pasta com o seu nome e data 

rf_query.to_excel('rolling_forecast.xlsx', index=False, sheet_name='Rolling forecast')

In [71]:
#Log de Premissas
    #Garantir que os percentuais não estão somando de forma que duplique ou remova valores


log = reprogramacao_melt.groupby(['Operação','Equipe atual', 'Mês Operacional'])['Valores'].sum().reset_index()

#Removendo todos os valores que são diferentes de 1
log = log[log['Valores'] != 1]


#Transformando  o log em um dicionário para incluir ele por inteiro no log de premissas

log_dict = log.to_dict(orient='records')

#Nome do arquivo
log_erros = 'log de erros.txt'


#Hora de execussão
hr_ex = datetime.now().strftime('%H:%M:%S')

l = '_'*150


#Gerando o arquivo com diferentes percentuais

with open (log_erros,'w', encoding= 'latin-1') as log_txt:
    log_txt.write('Log de erros:'+ '\n')
    log_txt.write('Operações com percentual diferente de zero que precisam de revisão'+'\n')
    log_txt.write("Se o campo 'Valores' for maior do que 1, logo é necessário que haja revisão dos percentuais atuais" +'\n')
    log_txt.write(f'Executado as {hr_ex}')

    log_txt.write('\n' + l + '\n')

    for record in log_dict:
        line = ", ".join([f"{k}: {v}" for k, v in record.items()])
        log_txt.write(line + "\n")


